### Pipeline OCR (Tesseract + OpenCV)28.

In [ ]:
import cv2
import pytesseract
import numpy as np
import os
from pdf2image import convert_from_path
from pyzbar.pyzbar import decode
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language


INPUT_FILE = "image2tab.webp"  
 
 
# SPACY 
import spacy 
from spacy.language import Language 
from spacy_langdetect import LanguageDetector 

nlp = spacy.load("en_core_web_sm") 

def get_lang_detector(nlp, name): 
    return LanguageDetector() 

# Enregistrer 
if not Language.has_factory("language_detector"):
    Language.factory("language_detector", func=get_lang_detector)

# Ajouter le pipe
if "language_detector" not in nlp.pipe_names:
    nlp.add_pipe("language_detector", last=True)


#  LOAD 
def load_input(path):
    if path.lower().endswith(".pdf"):
        pages = convert_from_path(path, dpi=200)
        imgs = []
        for p in pages:
            imgs.append(cv2.cvtColor(np.array(p), cv2.COLOR_RGB2BGR))
        return imgs
    return [cv2.imread(path)]

#  LANGUAGE 
def detect_language(text):
    doc = nlp(text)
    lang = doc._.language["language"]
    if lang not in ["fr", "en"]:
        lang = "en"
    return lang

#  OCR 
def ocr(img):
    return pytesseract.image_to_string(
        img,
        lang="fra+eng",
        config="--psm 3 --oem 3"
    )

#  MAIN 
imgs = load_input(INPUT_FILE)

for img in imgs:
    print("\n============")

    text = ocr(img)
    lang = detect_language(text)

    print("LANGUE :", lang)
    print("\n===== TEXTE COMPLET =================\n")
    print(text)



LANGUE : fr

===== TEXTE COMPLET =================

e

enireprise
A

—S

Ma petite

entreprise

19, rue de place 1° mai

FACTURE
CODE CLENT DATE
FCO01 4/20/2016 0002

CLIENT

SARL EL HANA

16000 Alger Centre IROUTE DE BEJAIA SETIF

Tel : 00-00-52-12- 19000 _ _ _

Ident Fiscal : 1601000000000

N°art : 16010000000

Mode de paiement : Espèce

Date Échéance : 5/20/2016

Référence Description Produit Quantité | P.Unitaire Valeur

001 [Produit 1 1000 100 1,000.00
1002 — [Produit 2 001 2.00 2,002.00
1003 — [Produit 3 1002 3.00 3,006.00
‘004 [Produit 1003 4.00 4,012.00
1005 [Produits 1004 5.00 5,020.00
1006 — |Produit 6 1005 6.00 6,030.00
1007 [Produit 7 1006 1100 11,066.00
1008 [Produit 8 1007 118.00 118,826.00
c1009 — [Produit 9 1008 19.00 19,152.00
1010 [Produit 10 1009 10.00 10,090.00

Non assujett &latva [Montant à payer 180,204.00

[Timbre 1,802.00
[Montant à payer tte 182,006.00

Monatnt Facture en Lettre cing

cent quatre vingt huit Dinars Algériens

Cachet & Signature



### Pipeline SpaCy de base & Tokenisation

In [8]:
from langdetect import detect
import spacy
import re

# Charger modèles une seule fois
nlp_fr = spacy.load("fr_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

texte = text  # vient de l'OCR

# Split phrases (plus propre que re.split simple)
sent_split = re.compile(r'(?<=[.!?])\s+')

for phrase in sent_split.split(texte):
    phrase = phrase.strip()

    # Ignorer vide / trop court
    if not phrase or len(phrase) < 20:
        continue

    # Lang detect (langdetect peut bugger sur texte sale)
    try:
        lang = detect(phrase)
    except:
        continue

    if lang == "fr":
        doc = nlp_fr(phrase)
    elif lang == "en":
        doc = nlp_en(phrase)
    else:
        continue

    print("\nPhrase :", phrase)
    print("Langue :", lang)
    print("Tokens :", [t.text for t in doc])



Phrase : e

enireprise
A

—S

Ma petite

entreprise

19, rue de place 1° mai

FACTURE
CODE CLENT DATE
FCO01 4/20/2016 0002

CLIENT

SARL EL HANA

16000 Alger Centre IROUTE DE BEJAIA SETIF

Tel : 00-00-52-12- 19000 _ _ _

Ident Fiscal : 1601000000000

N°art : 16010000000

Mode de paiement : Espèce

Date Échéance : 5/20/2016

Référence Description Produit Quantité | P.Unitaire Valeur

001 [Produit 1 1000 100 1,000.00
1002 — [Produit 2 001 2.00 2,002.00
1003 — [Produit 3 1002 3.00 3,006.00
‘004 [Produit 1003 4.00 4,012.00
1005 [Produits 1004 5.00 5,020.00
1006 — |Produit 6 1005 6.00 6,030.00
1007 [Produit 7 1006 1100 11,066.00
1008 [Produit 8 1007 118.00 118,826.00
c1009 — [Produit 9 1008 19.00 19,152.00
1010 [Produit 10 1009 10.00 10,090.00

Non assujett &latva [Montant à payer 180,204.00

[Timbre 1,802.00
[Montant à payer tte 182,006.00

Monatnt Facture en Lettre cing

cent quatre vingt huit Dinars Algériens

Cachet & Signature
Langue : fr
Tokens : ['e', '\n\n', 'enireprise', '\n', 'A'